# Mass reconstruction posibilities

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from tqdm import tqdm
import swordfish as sf
from WIMpy import DMUtils as DMU
from scipy.interpolate import interp1d
from scipy.integrate import cumtrapz
import h5py
from paleo.paleopy import *

rc('text', usetex=True)
rc('font',**{'family':'sans-serif','sans-serif':['cmr']})
rc('font',**{'family':'serif','serif':['cmr']})
%matplotlib inline

#### These are the bins we are considering - Change here

In [2]:
x_bins = np.linspace(1,1000,67)
x_width = np.diff(x_bins)
x_c = x_bins[:-1] + x_width/2
E_list = np.logspace(-1, 3, 500) # keV
nu_list = ['DSNB', 'atm', 'hep', '8B', '15O', '17F', '13N', 'pep','pp','7Be-384','7Be-861']

#### The region I will consider is between the 95% CL and three orders of magnitude above that. This roughly corresponds to the current sensitivity of Xenon detectors.

In [3]:
#### Lets first laod the backgrounds
dRdx_Sylnu = dRdx_nu(x_bins, E_list, rock='Syl', components=True, gaussian=True)
dRdx_Zabnu = dRdx_nu(x_bins, E_list, rock='Zab', components=True, gaussian=True)
bkgfis_Zab = fission_bkg(x_bins, 1, 1e7, rock='Zab', gaussian=True)
bkgfis_Syl = fission_bkg(x_bins, 1, 1e7, rock='Syl', gaussian=True)
dRdx_Sylnu.append(bkgfis_Syl)
dRdx_Zabnu.append(bkgfis_Zab)
nu_list.append('fission')

 DMutils.py: Loading neutrino flux for the first time...
Loading neutrino fluxes for...
    DSNB
    atm
    hep
    8B
    15O
    17F
    pep
    13N
    pp
    7Be-384
    7Be-861
...done.


#### Lets now make some Euclideanized signals

In [12]:
m, Syl_lim = np.loadtxt('../ES/limits/Sylvanite_lims_spectral10psys.txt', unpack=True)
lim = interp1d(m, Syl_lim, bounds_error=False, fill_value='extrapolate')

mlist = np.logspace(-0.5,4,num=100)
nsamples = 100
systematics = [0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.01]

couplings = []
mass = []
ESSyl = []
NSyl = [] 

for m in tqdm(mlist, desc="Euclideanizing Sylvanite"):
    coupling_temp = np.logspace(np.log10(lim(m)),np.log10(lim(m)*100), nsamples)
    for c_temp in coupling_temp:
        couplings.append(c_temp)
        mass.append(m)
        
        sig = gaussian_filter1d(dRdx(x_bins, c_temp, m, rock='Syl'),1)+1e-20
        SF = sf.Swordfish(dRdx_Zabnu, T=systematics, E=np.ones_like(dRdx_Zabnu[0])*10)
        
        NSyl.append(sum(sig))
        ES_temp = SF.euclideanizedsignal(sig)
        ESSyl.append(ES_temp)

couplings = np.array(couplings)
mass = np.array(mass)
ESSyl = np.array(ESSyl)
NSyl = np.array(NSyl)

# Output to new hdf5
outfile = '../ES/hdf5/Sylvanite.hdf5'
hf = h5py.File(outfile, 'w')
hf.create_dataset('ESSyl', data=ESSyl)
hf.create_dataset('mass', data=mass)
hf.create_dataset('c', data=couplings)
hf.create_dataset('NSyl', data=NSyl)
hf.close()

Euclideanizing Sylvanite:   3%|▎         | 3/100 [00:13<07:29,  4.64s/it]

KeyboardInterrupt: 

#### We can now try to assess how good at reconstruction Sylvanite is

In [4]:
root = h5py.File('../ES/hdf5/Sylvanite.hdf5')
ESSyl = np.array(root['ESSyl'])
sigmaSI = np.array(root['c'])
mass = np.array(root['mass'])
c = np.zeros([sigmaSI.shape[0], 2])
c[:,0] = mass
c[:,1] = sigmaSI
print(c.shape,ESsyl.shape)

(10000, 2) (10000, 66)


In [5]:
shSyl = sf.SignalHandler(c, ESSyl)

In [7]:
m_listSyl = []
sigma_list_Syl = []

for i in tqdm(range(len(c[:,0]))):
    P0 = c[i,:]
    pp, el_ind = shSyl.query_region(P0, sigma=2.0, d=1, return_indices = True)
    if pp.size == 0.0:
        continue
    if np.max(pp[:,0]) == np.max(mass):
        m_listSyl.append(c[i,0])
        sigma_list_Syl.append(c[i,1])

  0%|          | 0/10000 [00:00<?, ?it/s]/usr/local/lib/python3.6/site-packages/sklearn/linear_model/ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number2.335331e-20
  overwrite_a=True).T
100%|██████████| 10000/10000 [01:35<00:00, 104.76it/s]
